<a href="https://colab.research.google.com/github/heloisatambara/computacao-e-simulacao/blob/main/EP06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
from scipy.stats import chi2
from scipy.optimize import minimize_scalar
import numpy as np
import math 

In [139]:
# -*- coding: utf-8 -*-
"""
Resumo: Implementacao do algoritmo de Metropolis-Hastings

Autor: Eduardo Janotti Cavalcante         
         
Data: 20 de Maio de 2020
Ultima edicao: 15 de Julho de 2021

"""

def theta(theta1,theta3):
    # funcao potencial
    theta2 = 1 - theta1 - theta3
    if theta1 <0 or theta2<0 or theta3<0:
        return 0
    y = ((theta1**(x1-1))*(theta2**(x2-1))*(theta3**(x3-1)))
    return y
    

def alpha(i,j):
    # alpha de aceitacao utilizado
    if theta(i[0],i[1]) == 0: return 1
    return theta(j[0],j[1])/theta(i[0],i[1])


    
###############################################################################
    
    
def MCMC(aquecimento,tamanho_da_cadeia,saltos,mean,cov,ponto_inicial):
    #gera uma cadeia a partir do n do aquecimento, n do tamanho da cadeia e
    # tamanho dos saltos, matriz de covariancia e ponto inicial
    # funciona apenas para dimensao dois
    
    
    # total de pontos a serem gerados
    n = aquecimento+(tamanho_da_cadeia*saltos)
    
    
    # gera os valores da normal e uniforme anteriormente por vetorizacao
    normal = np.random.multivariate_normal(mean,cov,n)
    uniforme = np.random.uniform(0,1,n)
    
    #contagem dos pontos gerados para pegar o k-esimo candidato
    k = 0
    
    
    #cria a base da cadeia, apenas para facilitar o codigo
    cadeia = [0]*(tamanho_da_cadeia*saltos)

    
    
    
                        #### Aquecimento ####   
    # funcionamento igual ao topico MCMC, sem salvar a cadeia
    # Salvando apenas o ultimo membro
    cadeia_fria = ponto_inicial 
    for i in range(aquecimento):   
        atual = cadeia_fria
        proximo = (0,0)
        caminho = normal[k]
        proximo = (atual[0] + caminho[0] , atual[1] + caminho[1])
        if uniforme[k] < alpha(atual , proximo):
            cadeia_fria = proximo
        k += 1
        
        
        
    # calculo da taxa de aceitacao
    taxa_num = 0
    
            
            
                        #### MCMC ####   
        #######################################################################
    cadeia[0] = cadeia_fria        #A cadeia esquentada recebe o ultimo 
                                     #termo da fria
        #######################################################################
    for i in range((tamanho_da_cadeia*saltos)-1):     
        atual = cadeia[i]    # Posicao atual da cadeia
        #######################################################################
        # Calculo do proximo candidato
        proximo = (0,0)   
        caminho = normal[k]
        proximo = (atual[0] + caminho[0] , atual[1] + caminho[1])
        #######################################################################
                                                   # Decide se a nova posicao é
        if uniforme[k] < alpha(atual , proximo):    # aceita, movendo a 
            cadeia[i+1] = proximo                            # cadeia caso seja 
            
            taxa_num += 1 #calcula a taxa de aceitacao
            
        else:
            #repete o ultimo termo caso n seja aceito
            cadeia[i+1] = atual
            
        k += 1
        #######################################################################  
        
    #taxa de aceitacao
    taxa = taxa_num/len(cadeia)
        
    
    
    if saltos ==1:
        return cadeia,taxa
    
    # elimina parte da amostra para diminuir a correlacao
    cadeia_final = [0]*tamanho_da_cadeia
    for i in range(tamanho_da_cadeia):
        cadeia_final[i] = cadeia[saltos*i]
    return cadeia_final,taxa # a cadeia final é s?

def u(c,v):
      # calculo de u a partir da cadeia e de um vetor v de cortes, retorna vetor
      soma = [0]*len(v)
      for j in range(len(v)):
          # aqui q entra a ctt de integracao
          k = v[j]/(math.gamma(x1+x2+x3)/(math.gamma(x1)*math.gamma(x2)*math.gamma(x3)))
          for i in c:
              if theta(i[0],i[1]) > k:
                  soma[j] += 1
      somas = np.array(soma)
      return 1 - (somas/len(c) )



    

In [140]:
def getThetaVector(theta):
  thetaVector = [theta, 1 - theta - (1-theta**(1/2))**2, (1-theta**(1/2))**2]
  return thetaVector

  
def fdps(theta): # função densidade probabilidade - função S a ser maximizada 
    betha = ( math.factorial(x[0]+x[1]+x[2]-1) ) / ( math.factorial(x[0]-1)*math.factorial(x[1]-1)*math.factorial(x[2]-1) )
    thetaVector = [theta, 1 - theta - ((1-math.sqrt(theta))**2), (1-math.sqrt(theta))**2] # Hipotese de Hardy-Weinberg
    if thetaVector[0]<0 or thetaVector[1]<0 or thetaVector[2]<0: #
       return 0
    else:
      return -((thetaVector[0]**(x[0]-1)*thetaVector[1]**(x[1]-1)*thetaVector[2]**(x[2]-1))*betha) # negativo pois a função minimiza seu valor
     

def listOfXvectors(): # cria a lista dos vetores x a serem testados, mostrados no artigo
    Xvectors = []
    for x3 in range(2, 19):
        Xvectors.append([1, 19-x3, x3])
    for x3 in range(0,11):
        Xvectors.append([5, 15-x3, x3])
    for x3 in range(0,8):
        Xvectors.append([9, 11-x3, x3])
    return Xvectors

def sev(t,h,ev): # calcula o evalor normalizado
    y = 1- QQ(t,h,1-ev)
    return y

t = 2 #dim(theta) - seria 3 mas theta2 = n-theta1-theta3
h = 1 #dim(H) - seria 2 mas theta2 = n-theta1-theta3

def QQ(t,h,z): 
    fz = chi2.cdf( chi2.ppf(z,t), t-h)
    return fz



In [143]:
x = [1,17,2] # teste
a = minimize_scalar(fdps, bounds = (0,1), method = 'bounded')
print(a)
print(a.get('x'))
print(a.get('fun'))
print(fdps(a.get('x')))
print(u(c, [s_estrela]))

     fun: -0.19801676551659494
 message: 'Solution found.'
    nfev: 11
  status: 0
 success: True
       x: 0.22562494241741132
0.22562494241741132
-0.19801676551659494
-0.19801676551659494
[0.00348]


In [144]:
num = 100000  #numero de pontos

# media da dist normal utilizada
mean = [0,0]
# definificao da matriz de covariancia utilizada
# dimensao 2 pois o terceiro valor eh um menos os outros dois
sigma = 0.02
cov = np.array([[sigma,0],[0,sigma]])
#Ponto inicial da cadeia
ponto_inicial = (0,0)
saltos = 10
n_cadeia_fria = 10000

In [150]:
Y = [1,1,1] # priori
Xvectors = listOfXvectors() # gera todos os X na tabela
for X in Xvectors: # para cada vetor X:
    x1, x2, x3 = X[0]+1, X[1]+1, X[2]+1  # pega cada valor do vetor
    X[0], X[1], X[2] = x1, x2, x3  # X + Y
    c,taxa = MCMC(n_cadeia_fria,num,saltos,mean,cov,ponto_inicial) # gera a amostra
    maximizando = minimize_scalar(fdps, bounds = (0,1), method = 'bounded')
    s_estrela = -maximizando.get('fun') # acha o valor máximo de s
    ev = u(c, [s_estrela]) # acha W(s*) = ev(H|x) - e valor
    vetor_theta = getThetaVector(maximizando.get('x')) # acha o vetor theta
    Sev = sev(t,h,ev) # calcula e valor padronizado
    if Sev <= .05: H = "Rejeita H0" # decide se aceita ou rejeita H0
    else: H = "Aceita  H0"
    print(f" x1={x1-1} , x3={x3-1}, H={H},  θ*= {round(vetor_theta[0],4), round(vetor_theta[1],4), round(vetor_theta[2],4)}, ev(H|X)={format(ev[0],'.5f')} e sev(H|X)={format(Sev[0],'.5f')}")

 x1=1 , x3=2, H=Rejeita H0,  θ*= (0.2256, 0.4987, 0.2756), ev(H|X)=0.00345 e sev(H|X)=0.00076
 x1=1 , x3=3, H=Rejeita H0,  θ*= (0.2025, 0.495, 0.3025), ev(H|X)=0.01300 e sev(H|X)=0.00321
 x1=1 , x3=4, H=Rejeita H0,  θ*= (0.1806, 0.4888, 0.3306), ev(H|X)=0.03982 e sev(H|X)=0.01112
 x1=1 , x3=5, H=Rejeita H0,  θ*= (0.16, 0.48, 0.36), ev(H|X)=0.09051 e sev(H|X)=0.02838
 x1=1 , x3=6, H=Aceita  H0,  θ*= (0.1406, 0.4688, 0.3906), ev(H|X)=1.00000 e sev(H|X)=1.00000
 x1=1 , x3=7, H=Aceita  H0,  θ*= (0.1225, 0.455, 0.4225), ev(H|X)=0.31493 e sev(H|X)=0.12848
 x1=1 , x3=8, H=Aceita  H0,  θ*= (0.1056, 0.4387, 0.4556), ev(H|X)=0.48379 e sev(H|X)=0.22817
 x1=1 , x3=9, H=Aceita  H0,  θ*= (0.09, 0.42, 0.49), ev(H|X)=0.66624 e sev(H|X)=0.36747
 x1=1 , x3=10, H=Aceita  H0,  θ*= (0.0756, 0.3987, 0.5256), ev(H|X)=0.83342 e sev(H|X)=0.54605
 x1=1 , x3=11, H=Aceita  H0,  θ*= (0.0625, 0.375, 0.5625), ev(H|X)=0.95209 e sev(H|X)=0.75401
 x1=1 , x3=12, H=Aceita  H0,  θ*= (0.0506, 0.3487, 0.6006), ev(H|X)=0.999

In [151]:
Y = [0,0,0] # priori
Xvectors = listOfXvectors() # gera todos os X na tabela
for X in Xvectors: # para cada vetor X:
    x1, x2, x3 = X[0], X[1], X[2]  # pega cada valor do vetor
    X[0], X[1], X[2] = x1, x2, x3  # X + Y
    try:
      c,taxa = MCMC(n_cadeia_fria,num,saltos,mean,cov,ponto_inicial) # gera a amostra
      maximizando = minimize_scalar(fdps, bounds = (0,1), method = 'bounded')
      s_estrela = -maximizando.get('fun') # acha o valor máximo de s
      vetor_theta = getThetaVector(maximizando.get('x'))
      ev = u(c, [s_estrela]) # acha W(s*) = ev(H|x)
      Sev = sev(t,h,ev)
      if Sev <= .05: H = "Rejeita H0"
      else: H = "Aceita H0"
      print(f" x1={x1} , x3={x3}, H={H},  θ*= {round(vetor_theta[0],4), round(vetor_theta[1],4), round(vetor_theta[2],4)}, ev(H|X)={format(ev[0],'.5f')} e sev(H|X)={format(Sev[0],'.5f')}")
    except:
      print(f"x1={x1}, x3={x3}, Y= {Y} - não calculados, erro de domínio")

 x1=1 , x3=2, H=Rejeita H0,  θ*= (0.2215, 0.4983, 0.2803), ev(H|X)=0.00034 e sev(H|X)=0.00006
 x1=1 , x3=3, H=Rejeita H0,  θ*= (0.1946, 0.4931, 0.3123), ev(H|X)=0.00207 e sev(H|X)=0.00044
 x1=1 , x3=4, H=Rejeita H0,  θ*= (0.1695, 0.4844, 0.346), ev(H|X)=0.00770 e sev(H|X)=0.00181
 x1=1 , x3=5, H=Rejeita H0,  θ*= (0.1462, 0.4723, 0.3815), ev(H|X)=0.02354 e sev(H|X)=0.00618
 x1=1 , x3=6, H=Rejeita H0,  θ*= (0.1246, 0.4567, 0.4187), ev(H|X)=0.05458 e sev(H|X)=0.01588
 x1=1 , x3=7, H=Rejeita H0,  θ*= (0.1047, 0.4377, 0.4576), ev(H|X)=0.10703 e sev(H|X)=0.03451
 x1=1 , x3=8, H=Aceita H0,  θ*= (0.0865, 0.4152, 0.4983), ev(H|X)=0.18755 e sev(H|X)=0.06731
 x1=1 , x3=9, H=Aceita H0,  θ*= (0.0701, 0.3893, 0.5407), ev(H|X)=0.29698 e sev(H|X)=0.11917
 x1=1 , x3=10, H=Aceita H0,  θ*= (0.0554, 0.3599, 0.5848), ev(H|X)=0.42899 e sev(H|X)=0.19325
 x1=1 , x3=11, H=Aceita H0,  θ*= (0.0424, 0.327, 0.6306), ev(H|X)=0.56441 e sev(H|X)=0.28482
 x1=1 , x3=12, H=Aceita H0,  θ*= (0.0311, 0.2907, 0.6782), ev(H|